## Graphing in notebooks w/ matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
plt.plot([0,2],[2,4])
plt.axis(xmin=0,xmax=2,ymin=0,ymax=4)
plt.title("You can graph things inline!")
plt.xlabel("x axis")
plt.ylabel("y axis")
plt.show() # this is not necessary for notebooks, but is when you run the python script

## Drawing graphs
networkx is integrated with matplotlib, so you can draw graphs in notebooks

In [ ]:
import jellyfish.graphs as graphs
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import style
import numpy as np

In [ ]:
# This is a handy thing to use if you're making changes to a source file
import importlib
importlib.reload(graphs)

G = graphs.complete(10)
nx.draw(G)

# networkx uses matplotlib to draw things, 
# so you can save the image like this
plt.savefig("../figures/complete_graph.png")

plt.show()

In [ ]:
# here's what a fat tree looks like
G = graphs.fat_tree(4)
nx.draw(G)
plt.savefig("../figures/figure_1a.png")
plt.show()

In [ ]:
G = graphs.jellyfish(20, 4, 1)
pos = nx.spring_layout(G)
switches = [node for node in G.nodes() if "switch" in node]
hosts = [node for node in G.nodes() if "host" in node]
nx.draw_networkx_nodes(G, pos, nodelist=switches, node_color='b')
nx.draw_networkx_nodes(G, pos, nodelist=hosts, node_color='r')
nx.draw_networkx_edges(G, pos)

In [ ]:
for switch in switches: 
    print("{} connected to: {}".format(switch, G.adj[switch]))

In [ ]:
for host in hosts: 
    print("{} connected to: {}".format(host, G.adj[host]))

In [ ]:
import importlib
importlib.reload(graphs)

In [ ]:
G = graphs.jellyfish(16,4,1)

In [ ]:
import importlib
importlib.reload(graphs)

In [ ]:
#this part is for 1c 
# k = 14, k-r = 3, total servers = 245
fg = graphs.fat_tree(14)
jg = graphs.jellyfish(98, 14, 7)

jg_switches = [node[0] for node in list(jg.nodes(data=True)) if node[1]['type'] == 'switch']
jg_hosts = [node[0] for node in list(jg.nodes(data=True)) if node[1]['type'] == 'host']

jg_shortest_paths = []

for jg_src_host in jg_hosts: 
    for jg_dst_host in jg_hosts: 
        if jg_src_host == jg_dst_host: 
            continue 
        else: 
            jg_shortest_paths.append(nx.shortest_path_length(jg, jg_src_host, jg_dst_host))

print(len(jg_shortest_paths))

fg_switches = []
for node in list(fg.nodes(data=True)):
    if node[1]['type'] == 'switch': 
        fg_switches.append(node[0])

fg_hosts = []
for node in list(fg.nodes(data=True)):
    if node[1]['type'] == 'host': 
        fg_hosts.append(node[0])

fg_shortest_paths = []

for fg_src_host in fg_hosts: 
    for fg_dst_host in fg_hosts: 
        if fg_src_host == fg_dst_host: 
            continue 
        else: 
            fg_shortest_paths.append(nx.shortest_path_length(fg, fg_src_host, fg_dst_host))
            
print(len(fg_shortest_paths))

fg_frequencies = [fg_shortest_paths.count(i)/len(fg_shortest_paths) for i in range(2,7,1)]
print(fg_frequencies)

jg_frequencies = [jg_shortest_paths.count(i)/len(jg_shortest_paths) for i in range(2,7,1)]
print(jg_frequencies)

style.use('seaborn')
# inspired from https://matplotlib.org/examples/api/barchart_demo.html 
# and https://matplotlib.org/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py

labels = ['2', '3', '4', '5', '6']
n = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
bars_fg = ax.bar(n + width/2, fg_frequencies, width=0.35, label="Fat Tree", color='red')
bars_jg = ax.bar(n - width/2, jg_frequencies, width=0.35, label="Jellyfish", color='blue')
ax.set_xlabel("Path Length")
ax.set_ylabel("Fraction of Server Pairs")
ax.set_xticks(n)
ax.set_yticks(np.arange(10)/10)
ax.set_xticklabels(labels)
ax.legend( (bars_fg[0], bars_jg[0]), ("Fat Tree", "Jellyfish"))
plt.show()

In [ ]:
def approx_bb(N, k, kind='fat_tree'):
    num_switches = Ny 
    degree = k 
    
    if(kind == 'fat_tree'):         
        total_hosts = (k**3)/4 #this is not varied for fat tree topology in the paper
        bisection_bandwidth = (k**3)/8 
        
        # assume all links have same line rate
        host_to_switch_bandwidth_per_bisection = total_hosts/2
        
        # we calculatre normalized bisection bandwidth as 
        # the number of links crossing the bisection divided by 
        # the number of links between servers and switches in one 
        # network partition. 
        # Inspired by: https://piazza.com/class/k7m53jbmw0n6qf?cid=63
        normalized_bb = bisection_bandwidth / host_to_switch_bandwidth_per_bisection
        return ([total_hosts/1000], [normalized_bb])
    
    if(kind == 'jellyfish'): 
        # we hold the number of switches constant
        # and vary the number of edges used for switch-switch
        # links and the number of edges used for switch-host 
        # links by changes num_hosts parameter 
        hosts = []
        normalized_bbs = []
        for num_hosts in range(1, k,): 
            r = k - num_hosts # k - (k - r) = r 
            
            # |S| = N/2 in the case of a bisection 
            delta_s = (1 - np.sqrt(np.log(2)/r)) * (r/2) * (N/2)
            
            # in expectation (N/2) switches are in a bisection partition 
            switch_to_host_links_in_partition = (N/2)*num_hosts
            
            normalized_bb = delta_s / switch_to_host_links_in_partition
            total_hosts = num_hosts * N
            
            hosts.append(total_hosts/1000)
            normalized_bbs.append(normalized_bb)
            
        return (hosts, normalized_bbs)

In [ ]:
style.use('seaborn')

jelly_720_24 = approx_bb(720, 24, kind = 'jellyfish')
jelly_1280_32 =approx_bb(1280, 32, kind = 'jellyfish')
jelly_2880_48 =approx_bb(2880, 48, kind = 'jellyfish')
fat_720_24 = approx_bb(720, 24, kind = 'fat_tree')
fat_1280_32 = approx_bb(1280, 32, kind = 'fat_tree')
fat_2880_48 = approx_bb(2880, 48, kind = 'fat_tree')


fig, ax = plt.subplots()
jelly_720_24_plot = ax.plot(jelly_720_24[0], jelly_720_24[1], label = "Jellyfish, N=720, k=24", color='purple', marker='^')
jelly_1280_32_plot = ax.plot(jelly_1280_32[0], jelly_1280_32[1], label = "Jellyfish, N=1280, k=32", color='limegreen', marker ='o')
jelly_2880_48_plot = ax.plot(jelly_2880_48[0], jelly_2880_48[1], label = "Jellyfish, N=2880, k=48", color='blue', marker = 's')
fat_720_24_plot = ax.scatter(fat_720_24[0], fat_720_24[1], label = "Fat Tree, N=720, k=24", color='black', marker='^', s=300)
fat_1280_32_plot = ax.scatter(fat_1280_32[0], fat_1280_32[1], label = "Fat Tree, N=1280, k=32", color='green', marker='o', s=300)
fat_2880_48_plot = ax.scatter(fat_2880_48[0], fat_2880_48[1], label = "Fat Tree, N=2880, k=48", color='red', marker='s', s=300)

ax.set_xlabel("Number of Servers in Thousands")
ax.set_ylabel("Normalized Bisection Bandwidth")
ax.set_xticks(np.arange(9)*10)
ax.set_yticks(np.arange(start=0.2, stop=1.8, step=0.2))
ax.set_xlim([0,80])
ax.set_ylim([0.2,1.6])
ax.legend(labelspacing = 1.25, frameon=True)
plt.show()

In [ ]:
def approx_num_servers(k, kind):
    if kind == 'fat_tree': 
        
        num_hosts = []
        num_ports = []
        
        for k in [24,32,48,64]:
            # https://packetpushers.net/demystifying-dcn-topologies-clos-fat-trees-part2/
            hosts = (k**3)/4
            switches = 5*((k/2)**2)
            ports = k * switches
            num_hosts.append(hosts/1000)
            num_ports.append(ports/1000)
        return (num_hosts, num_ports)
        
    if kind == 'jellyfish': 
        
        # search for lowest r value that fulfills bisection bandwidth such that r > 3
        r = 3
        while True: 
            # (delta_s)/((N/2) * (k-r)) = (1 - np.sqrt(np.log(2)/r)) * (r/2) * (N/2) = 1
            val = ((1 - np.sqrt(np.log(2)/r)) * (r/2))/(k-r)
            if val >= 1: 
                break
            else: 
                r += 1 
                
        num_hosts = []
        num_ports = []
        
        for hosts in range(8, 80000, 8): 
            switches = hosts/(k-r)
            ports = switches * k 
            num_hosts.append(hosts/1000)
            num_ports.append(ports/1000)
        return (num_hosts, num_ports)
            
        

In [ ]:
style.use('seaborn')
fat_tree = approx_num_servers(0, "fat_tree")
jelly_24 = approx_num_servers(24, "jellyfish")
jelly_32 = approx_num_servers(32, "jellyfish")
jelly_48 = approx_num_servers(48, "jellyfish")
jelly_64 = approx_num_servers(64, "jellyfish")

fig, ax = plt.subplots()
fat_tree_plot = ax.scatter(fat_tree[0], fat_tree[1], label = "Fat-tree; {23, 32, 48, 64} ports", color='red', marker='o')
jelly_24_plot = ax.plot(jelly_24[0], jelly_24[1], label = "Jellyfish; 24 ports", color='purple', lw = 0.5)
jelly_32_plot = ax.plot(jelly_32[0], jelly_32[1], label = "Jellyfish; 32 ports", color='green', lw = 4)
jelly_48_plot = ax.plot(jelly_48[0], jelly_48[1], label = "Jellyfish; 48 ports", color='blue', ls = ":", lw = 2)
jelly_64_plot = ax.plot(jelly_64[0], jelly_64[1], label = "Jellyfish; 64 ports", color='gold', lw = 3)

ax.set_xlabel("Number of Servers in Thousands")
ax.set_ylabel("Equipment Cost [#Ports in Thousands]")
ax.set_xticks(np.arange(8)*10)
ax.set_yticks(np.arange(start=0, stop=450, step=50))
ax.set_xlim([0,80])
ax.set_ylim([0, 400])
ax.legend(labelspacing = 1.25, frameon=True)
plt.show()

In [ ]:
#figure 9 

In [ ]:
G = graphs.jellyfish(98, 14, 7)

In [ ]:
def linkrank(n, degree, num_hosts):
    G = graphs.jellyfish(n, degree, num_hosts)
    import random
    import copy 

    random.seed(42)

    #randomly permute hosts into two groups and compare by index for efficiency 
    hosts = [node[0] for node in list(G.nodes(data=True)) if node[1]['type'] == 'host']
    hosts_a = copy.deepcopy(hosts)
    hosts_b = copy.deepcopy(hosts)
    random.shuffle(hosts_a)
    random.shuffle(hosts_b)

    ecmp8_edge_count = {}
    ecmp64_edge_count = {}
    k_shortest8_edge_count = {}

    def increment_path(path, edge_tup_to_count):
        for i in range(len(path)-1):
            tup = (path[i], path[i+1]) 
            if tup in edge_tup_to_count: 
                val = edge_tup_to_count[tup]
                val += 1 
                edge_tup_to_count[tup] = val 
            else: 
                edge_tup_to_count[tup] = 1

    for i in range(len(hosts)):
        src = hosts_a[i]
        dst = hosts_b[i]
        if src == dst: 
            continue

        paths = nx.algorithms.shortest_paths.generic.all_shortest_paths(G, src, dst)
        path_list = list(paths)

        if len(path_list) > 0:
            path = random.choice(path_list[:8])
            increment_path(path, ecmp8_edge_count)

            path = random.choice(path_list[:64])
            increment_path(path, ecmp64_edge_count)

        for path in path_list[:8]:
            increment_path(path, k_shortest8_edge_count)

    ecmp8_list = list(ecmp8_edge_count.values())
    ecmp64_list = list(ecmp64_edge_count.values())
    k_shortest8_list = list(k_shortest8_edge_count.values())

    # we must represent every link that was never used during the simulation 
    num_links = nx.number_of_edges(G)*2
    if len(ecmp8_list) < num_links: 
        for i in range(num_links - len(ecmp8_list)):
            ecmp8_list.append(0)
    if len(ecmp64_list) < num_links: 
        for i in range(num_links - len(ecmp64_list)):
            ecmp64_list.append(0)
    if len(k_shortest8_list) < num_links: 
        for i in range(num_links - len(k_shortest8_list)):
            k_shortest8_list.append(0)

    ecmp8_list = sorted(ecmp8_list)
    ecmp64_list = sorted(ecmp64_list)
    k_shortest8_list = sorted(k_shortest8_list)
    
    style.use('seaborn')

    x = np.arange(len(ecmp8_list))

    fig, ax = plt.subplots()
    ecmp8_plot = ax.plot(x, ecmp8_list, label = "8-way ECMP", color='green')
    ecmp64_plot = ax.plot(x, ecmp64_list, label = "64-way ECMP", color='red')
    k_shortest8_plot = ax.plot(x, k_shortest8_list, label = "8 Shortest Paths", color='blue', lw = 3)

    ax.set_xlabel("Rank of Link")
    ax.set_ylabel("# Distinct Paths Link is on")
    ax.set_xticks(np.arange(6)*500)
    ax.set_yticks(np.arange(start=0, stop=20, step=2))
    ax.set_xlim([0,3000])
    ax.set_ylim([0,18])
    ax.legend(labelspacing = 1.25, frameon=True)
    plt.show()


In [ ]:
linkrank(98, 14, 7)

In [ ]:
import random
import copy 

random.seed(42)

#randomly permute hosts into two groups and compare by index for efficiency 
hosts = [node[0] for node in list(G.nodes(data=True)) if node[1]['type'] == 'host']
hosts_a = copy.deepcopy(hosts)
hosts_b = copy.deepcopy(hosts)
random.shuffle(hosts_a)
random.shuffle(hosts_b)

ecmp8_edge_count = {}
ecmp64_edge_count = {}
k_shortest8_edge_count = {}

def increment_path(path, edge_tup_to_count):
    for i in range(len(path)-1):
        tup = (path[i], path[i+1]) 
        if tup in edge_tup_to_count: 
            val = edge_tup_to_count[tup]
            val += 1 
            edge_tup_to_count[tup] = val 
        else: 
            edge_tup_to_count[tup] = 1

for i in range(len(hosts)):
    src = hosts_a[i]
    dst = hosts_b[i]
    if src == dst: 
        continue
        
    paths = nx.algorithms.shortest_paths.generic.all_shortest_paths(G, src, dst)
    path_list = list(paths)

    if len(path_list) > 0:
        path = random.choice(path_list[:8])
        increment_path(path, ecmp8_edge_count)

        path = random.choice(path_list[:64])
        increment_path(path, ecmp64_edge_count)

    for path in path_list[:8]:
        increment_path(path, k_shortest8_edge_count)
        
ecmp8_list = list(ecmp8_edge_count.values())
ecmp64_list = list(ecmp64_edge_count.values())
k_shortest8_list = list(k_shortest8_edge_count.values())

# we must represent every link that was never used during the simulation 
num_links = nx.number_of_edges(G)*2
if len(ecmp8_list) < num_links: 
    for i in range(num_links - len(ecmp8_list)):
        ecmp8_list.append(0)
if len(ecmp64_list) < num_links: 
    for i in range(num_links - len(ecmp64_list)):
        ecmp64_list.append(0)
if len(k_shortest8_list) < num_links: 
    for i in range(num_links - len(k_shortest8_list)):
        k_shortest8_list.append(0)
        
ecmp8_list = sorted(ecmp8_list)
ecmp64_list = sorted(ecmp64_list)
k_shortest8_list = sorted(k_shortest8_list)

In [ ]:
style.use('seaborn')

x = np.arange(len(ecmp8_list))

fig, ax = plt.subplots()
ecmp8_plot = ax.plot(x, ecmp8_list, label = "8-way ECMP", color='green')
ecmp64_plot = ax.plot(x, ecmp64_list, label = "64-way ECMP", color='red')
k_shortest8_plot = ax.plot(x, k_shortest8_list, label = "8 Shortest Paths", color='blue', lw = 3)

ax.set_xlabel("Rank of Link")
ax.set_ylabel("# Distinct Paths Link is on")
ax.set_xticks(np.arange(6)*500)
ax.set_yticks(np.arange(start=0, stop=20, step=2))
ax.set_xlim([0,3000])
ax.set_ylim([0,18])
ax.legend(labelspacing = 1.25, frameon=True)
plt.show()

In [ ]:
G = graphs.jellyfish(98,14,7)

In [ ]:
list(G.edges())

In [ ]:
G = graphs.jellyfish()

In [ ]:
from jellyfish import mininet

In [ ]:
a = mininet.graph_to_topo(graphs.jellyfish(20,4,1))

In [ ]:
print(type(a))

In [ ]:
mininet.make_mininet(graphs.jellyfish(20,4,1))

In [ ]:
import matplotlib.pyplot as plt
import jellyfish.graphs as graphs
import networkx as nx
import numpy as np
from matplotlib import style
import random
import os
import sys
import shutil
import jellyfish

import mininet
import mininet.clean
from mininet.net import Mininet
from mininet.cli import CLI
from mininet.log import lg, info
from mininet.link import TCLink, Link, TCIntf
from mininet.node import Node, OVSKernelSwitch, RemoteController
from mininet.topo import Topo
from mininet.util import waitListening,custom
import math

In [ ]:
#1c mininet 

In [ ]:
jelly_ping_vals = [77.153, 24.453999999999997, 76.317, 54.486, 23.165000000000003, 26.871, 56.580999999999996, 34.138, 26.098, 121.74399999999999, 37.488, 48.149, 75.61, 17.433, 37.829, 31.264, 96.92, 170.554, 44.127, 97.516, 120.23, 33.105, 29.923, 96.544, 34.601, 137.638, 106.437, 33.763, 23.223, 121.65500000000002, 80.163, 28.320000000000004, 65.573, 82.975, 44.228, 49.556999999999995, 40.388, 38.577, 25.586, 30.331, 68.112, 11.906, 17.847, 40.405, 90.731, 67.026, 39.603, 26.916, 157.795, 42.842000000000006, 37.43, 51.893, 46.37199999999999, 83.199, 116.447, 62.978, 33.306, 18.341, 42.439, 35.291, 71.157, 75.663, 30.366, 94.927, 90.509, 71.244, 39.395, 30.694, 22.763, 86.127, 29.286, 11.217999999999998, 43.080000000000005, 38.329, 72.673, 79.308, 21.532, 59.058, 44.317, 41.971, 38.199, 27.316999999999997, 28.755, 9.964, 16.998, 42.655, 9.307, 45.781, 43.1, 41.843, 110.2, 37.022, 45.798, 61.405, 59.338, 75.215, 100.88600000000001, 16.334, 23.115, 34.733, 12.484, 64.053, 39.555, 17.291, 86.149, 64.153, 53.119, 54.98, 44.839, 58.84, 98.163, 61.183, 44.183, 20.629, 62.517, 61.076, 9.914, 43.185, 49.422, 86.26100000000001, 52.44899999999999, 76.363, 60.794000000000004, 36.268, 42.679, 20.983, 7.905, 30.925, 26.209999999999997, 65.125, 45.455000000000005, 27.176, 32.761, 44.25, 29.004, 48.865, 44.489, 65.544, 118.46499999999999, 22.467, 11.42, 20.769, 25.425, 36.906, 25.321, 19.122, 26.491, 32.422, 18.17, 28.852999999999998, 25.098, 48.796, 92.11, 35.003, 26.993, 21.802000000000003, 32.784, 68.982, 24.888, 35.699, 44.262, 33.459, 23.774999999999995, 14.672000000000002, 20.638, 41.394, 74.813, 23.0, 14.31, 10.896, 23.565, 72.451, 23.899, 27.201000000000004, 35.249, 30.719000000000005, 22.916, 12.283, 10.553, 31.735, 66.661, 42.989, 33.28, 14.421, 49.435, 17.388, 10.534, 49.707, 47.749, 14.569, 41.35, 22.611, 16.482, 13.744, 46.104000000000006, 62.363, 53.66299999999999, 31.790000000000003, 66.837, 27.945000000000004, 12.437, 48.431999999999995, 58.592000000000006, 22.761, 41.34, 38.742, 37.976, 37.035, 15.967999999999998]
#jelly_ping_vals_b = [2140.663, 40.452, 64.207, 74.843, 76.661, 21.548999999999996, 46.669000000000004, 55.258, 61.667, 43.48499999999999, 54.151, 36.756, 50.36600000000001, 59.409, 30.307, 44.014, 85.142, 59.609, 75.511, 144.221, 48.008, 211.891, 33.42, 25.258, 84.742, 29.576000000000004, 54.483, 51.058, 15.627, 18.094, 87.442, 63.17600000000001, 54.204, 19.094, 32.133, 42.863, 39.498, 24.556, 25.588000000000005, 14.684, 19.243, 34.206, 49.344, 23.887, 22.935999999999996, 78.813, 50.737, 55.816, 55.351, 41.206, 55.364, 45.57, 28.407, 35.64, 55.463, 29.929, 32.259, 21.56, 17.106, 50.389, 35.63, 70.569, 85.474, 29.435999999999996, 82.415, 44.638, 39.503, 35.076, 65.848, 35.877, 20.857, 23.828, 22.42, 52.705999999999996, 13.069, 80.003, 75.179, 29.133, 52.469, 44.453, 36.841, 55.233, 46.222, 33.978, 9.092, 30.866, 12.661, 54.095, 37.904, 35.741, 28.45, 116.955, 49.07500000000001, 112.07, 41.825, 30.589, 30.838000000000005, 68.888, 16.293, 26.749, 12.632, 44.685, 32.521, 39.686, 12.142000000000001, 61.92600000000001, 66.646, 52.282000000000004, 59.85999999999999, 50.91400000000001, 44.728, 44.927, 20.04, 19.277, 19.404, 67.141, 11.793, 10.522, 25.346, 39.879, 61.582, 37.234, 18.298, 32.059, 45.30100000000001, 27.267, 38.522, 13.708999999999998, 18.61, 55.080999999999996, 36.686, 27.507, 27.812, 34.8, 34.911, 24.471, 47.779, 33.861, 60.149, 53.943999999999996, 27.356000000000005, 10.105, 16.481, 37.841, 23.778000000000002, 20.31, 23.782, 28.350000000000005, 21.346999999999998, 9.337, 34.776, 23.052000000000003, 39.544, 54.002, 33.754, 26.820000000000004, 22.665000000000003, 35.298, 16.44, 17.368, 18.748, 21.624, 10.296, 22.261, 17.829, 25.947, 43.476, 18.361, 20.798, 15.424, 10.395, 37.448, 24.871, 20.021, 18.097, 16.965, 14.035000000000002, 32.045, 24.477999999999998, 17.366, 33.859, 46.117, 19.854, 26.006, 10.776000000000002, 56.043, 38.459, 32.877, 14.901000000000002, 18.643, 29.244, 31.131, 26.137, 27.372, 16.793, 62.023, 37.756, 32.961, 20.393, 13.488, 18.11, 18.114, 18.657, 23.504, 12.175000000000002, 23.996, 16.796, 9.914, 18.964, 26.753]
fat_ping_vals = [48.209, 27.024999999999995, 32.892, 51.214, 44.328, 40.539, 49.10999999999999, 44.769, 42.278, 54.977, 49.334, 53.21, 52.16, 54.770999999999994, 16.077, 32.133, 28.667999999999996, 62.281, 46.043, 47.579, 58.352, 47.347, 46.278, 41.48, 46.519, 45.096, 47.773, 57.94, 57.83599999999999, 15.564, 18.714, 9.256, 74.347, 74.653, 80.419, 64.45, 47.818000000000005, 43.306999999999995, 70.066, 51.97, 62.37, 69.011, 55.704, 55.055, 14.619, 13.836999999999998, 9.062, 141.364, 59.232, 54.657999999999994, 64.493, 48.286, 51.567, 56.446, 60.849, 58.763, 54.342999999999996, 61.978, 103.88, 41.53, 32.264, 50.623, 30.532, 9.667, 44.154, 30.631, 78.181, 94.51, 85.092, 72.229, 37.743, 39.831, 49.918, 56.18699999999999, 23.712, 26.443, 48.166, 49.458, 3.9, 40.963, 30.036, 156.629, 167.248, 105.972, 86.20300000000002, 57.589, 117.021, 43.296, 44.689, 26.661999999999995, 76.221, 32.102, 36.326, 17.766, 16.6, 13.316, 195.764, 100.68599999999999, 193.071, 99.23200000000001, 146.201, 62.792, 85.182, 122.661, 36.888, 24.713000000000005, 222.667, 124.615, 71.096, 28.171000000000003, 10.413, 478.75, 346.88899999999995, 316.163, 247.929, 103.476, 116.445, 87.36899999999999, 97.568, 81.912, 47.507, 26.318, 33.112, 177.02699999999996, 106.27500000000002, 104.286, 64.043, 15.21, 62.314, 63.216, 150.312, 156.323, 139.309, 194.58, 59.69899999999999, 41.995, 37.221, 31.682, 63.471, 72.327, 72.478, 62.392, 5.813, 56.625, 55.144, 130.945, 168.303, 112.455, 123.544, 32.816, 34.09, 29.939000000000004, 28.064000000000004, 58.132, 67.571, 44.431000000000004, 52.669000000000004, 19.547, 22.693, 18.181, 173.11, 102.01299999999999, 90.221, 99.98200000000001, 38.567, 28.145, 26.375, 26.404, 43.386, 46.169999999999995, 41.356, 49.333, 13.716, 18.002, 7.766000000000001, 89.017, 123.466, 109.625, 112.359, 30.362, 29.396, 36.506, 100.08300000000001, 38.611, 29.385, 28.991, 29.471, 44.957, 49.524, 55.729000000000006, 48.488, 19.251, 38.459, 42.51, 32.917, 28.369, 48.47299999999999, 39.197, 30.689000000000004, 29.458, 28.028000000000002, 25.621, 51.693000000000005, 54.092000000000006, 49.257, 52.315, 5.2, 45.576, 33.575, 74.692, 27.343, 40.257, 41.505, 29.146, 25.396, 34.74, 26.754, 56.333, 51.931999999999995, 43.37199999999999, 52.144, 24.26, 17.833, 18.696, 31.046000000000003, 24.518999999999995, 36.906, 34.621, 33.949, 46.13399999999999, 42.66, 29.618, 54.617, 50.157999999999994, 55.74699999999999, 60.696, 23.389, 25.956, 6.773]






In [ ]:
# norm_a = a / np.sqrt(np.sum(a**2))
# norm_b = np.ceil(a/min(a))
# norm_a * 10
def norm_pings(pings):
    np_pings = np.array(pings)
    norm_pings = np_pings/np.min(np_pings)
    norm_pings = np.ceil(norm_pings)
    norm_pings = norm_pings.astype(int)
    norm_pings = list(norm_pings)
    return norm_pings 

jelly_norm_pings = norm_pings(jelly_ping_vals)
fat_norm_pings = norm_pings(fat_ping_vals)

In [ ]:
length = max(max(jelly_norm_pings), max(fat_norm_pings))
if length > 20: 
    length = 20

jelly_ping_freq = [jelly_norm_pings.count(i)/len(jelly_norm_pings) for i in range(length)]
fat_ping_freq = [fat_norm_pings.count(i)/len(fat_norm_pings) for i in range(length)]

In [ ]:
style.use('seaborn')
# inspired from https://matplotlib.org/examples/api/barchart_demo.html
# and https://matplotlib.org/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py

labels = [str(i) for i in range(length)]
n = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
bars_jelly = ax.bar(n - width/2, jelly_ping_freq, width=0.35, label="Jellyfish", color='blue')
bars_fat = ax.bar(n + width/2, fat_ping_freq, width=0.35, label="Fat Tree", color='red')
ax.set_xlabel("Path Length")
ax.set_ylabel("Fraction of Server Pairs")
ax.set_xticks(n)
ax.set_yticks(np.arange(10)/10)
ax.set_xticklabels(labels)
ax.legend()
plt.show()

In [1]:
import matplotlib.pyplot as plt
import jellyfish.graphs as graphs
import networkx as nx
import numpy as np
from matplotlib import style
import random
import os
import sys
import shutil
import jellyfish

import mininet
import mininet.clean
from mininet.net import Mininet
from mininet.cli import CLI
from mininet.log import lg, info
from mininet.link import TCLink, Link, TCIntf
from mininet.node import Node, OVSKernelSwitch, RemoteController
from mininet.topo import Topo
from mininet.util import waitListening,custom
import math
import re


In [2]:
#table 1

In [3]:
os.getcwd()

'/vagrant/notebooks'

In [4]:
mininet_results_dir = '/vagrant/mininet_results'
bandwidths = []

for file in os.listdir(mininet_results_dir):
    f = open(mininet_results_dir + '/' + file, 'r')
    for line in f: 
        data = f.read()
        if 'Mbits/sec' in data.split():
            index = data.split().index('Mbits/sec')
            num = float(data.split()[index-1])
            bandwidths.append(num)
bandwidths

[9.46, 10.7, 9.92, 9.61, 9.26, 10.1, 7.74, 8.94, 6.63, 9.54, 9.4, 10.2]

In [11]:
np_bandwidth = np.array(bandwidths)
norm_bd = np_bandwidth / np.sqrt(np.sum(np_bandwidth**2))
norm_avg_bd = np.mean(norm_bd)
norm_avg_bd

0.286788386498175

In [9]:
norm_avg_bd = sum(bandwidths)/len(bandwidths)
norm_avg_bd

9.291666666666666